In [0]:
# Notebook: 01_Data_Ingestion_Medallion.py

import pyspark.sql.functions as F

# --- 1. CONFIGURAÇÃO DO UNITY CATALOG E ARQUIVOS ---
UC_CATALOG = "dev_catalogue"
UC_SCHEMA = "staging_schema"
BRONZE_TABLE = f"{UC_CATALOG}.{UC_SCHEMA}.bronze_insurance_costs"

# CORREÇÃO: Usando o caminho do Volume do Unity Catalog
# ASSUMINDO que o arquivo 'insurance.csv' está dentro deste Volume.
CSV_FILE_PATH = "/Volumes/dev_catalogue/staging_schema/raw_data_volume/insurance.csv" 


# Cria o schema se não existir (garantia)
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {UC_CATALOG}.{UC_SCHEMA}")
print(f"Schema {UC_CATALOG}.{UC_SCHEMA} verificado.")


# --- 2. LEITURA DO ARQUIVO BRUTO A PARTIR DO VOLUME ---
print(f"Lendo dados brutos de: {CSV_FILE_PATH}")

# Leitura do CSV: infere o esquema e considera o cabeçalho
df_raw = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(CSV_FILE_PATH)
)

print("\nPrimeiras 5 linhas do DataFrame BRONZE lido do Volume:")
df_raw.show(5)
print(f"Esquema inferido: {df_raw.printSchema()}")

# --- 3. SALVAMENTO NA CAMADA BRONZE (DELTA LAKE) ---
# Reescrevemos a tabela Bronze. Como ela já existe, o modo 'overwrite' a atualiza.
df_raw.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(BRONZE_TABLE)

print("-" * 50)
print(f"✅ Camada BRONZE atualizada com sucesso na Tabela Delta: {BRONZE_TABLE}")
print("-" * 50)

In [0]:
# Célula 2: Criação da Camada SILVER (Limpeza e Chaves)

from pyspark.sql.functions import monotonically_increasing_id, col, regexp_replace

# --- 1. CONFIGURAÇÃO DE CAMINHOS ---
BRONZE_TABLE = "dev_catalogue.staging_schema.bronze_insurance_costs"
SILVER_TABLE = "dev_catalogue.staging_schema.silver_insurance_features"

# Leitura da Tabela Bronze
df_silver = spark.read.table(BRONZE_TABLE)

print(f"Lido {df_silver.count()} registros da Camada BRONZE.")

# --- 2. PADRONIZAÇÃO E LIMPEZA BÁSICA ---
# 2.1 Padronizar nomes de colunas (Snake_Case)
df_silver = df_silver.select(
    col("age"),
    col("sex"),
    col("bmi"),
    col("children"),
    col("smoker"),
    # Padronizar a coluna 'region'
    regexp_replace(col("region"), "-", "_").alias("region"), 
    # Renomear 'charges' para 'insurance_cost' (melhor para BI)
    col("charges").alias("insurance_cost")
)

# 2.2 Tratamento de nulos (Não esperado neste dataset, mas boa prática)
# Exemplo: remover linhas onde a coluna principal 'insurance_cost' for nula.
df_silver = df_silver.filter(col("insurance_cost").isNotNull())


# --- 3. CRIAÇÃO DE CHAVES SUBSTITUTAS (SURROGATE KEYS) ---
# O 'monotonically_increasing_id' é usado para gerar chaves exclusivas.
# Essas chaves serão as Pks das Dimensões e Fks da Fato.

df_silver = df_silver.withColumn(
    "SK_ID", # Chave Primária Substituta
    monotonically_increasing_id()
)

# Criar chaves substitutas para as dimensões:
# Nota: Usamos a mesma SK_ID para todas as Dimensões que se relacionarão
# diretamente com a Tabela Fato no Esquema Estrela.

df_silver = df_silver.withColumn(
    "SK_LOCATION",
    monotonically_increasing_id()
).withColumn(
    "SK_HABIT",
    monotonically_increasing_id()
).withColumn(
    "SK_TIME",
    F.lit(20250101).cast("int") # Chave fixa temporária, pois a data não está no dataset
)


# --- 4. SELEÇÃO FINAL DA CAMADA SILVER ---
# Selecionamos todas as colunas essenciais, incluindo as novas chaves
df_silver_final = df_silver.select(
    "SK_ID",
    "SK_LOCATION",
    "SK_HABIT",
    "SK_TIME",
    
    # Atributos (Features)
    "age",
    "sex",
    "bmi",
    "children",
    "smoker",
    "region",
    
    # Métrica (Label)
    "insurance_cost"
)

# --- 5. SALVAR NA CAMADA SILVER ---
df_silver_final.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(SILVER_TABLE)

print("\nPrimeiras 5 linhas da Camada SILVER (Com Chaves SK):")
df_silver_final.show(5)
print(f"Total de registros na SILVER: {df_silver_final.count()}")
print(f"✅ Camada SILVER (Limpa e com SKs) salva com sucesso em: {SILVER_TABLE}")

In [0]:
# Célula 3: Criação da Camada GOLD (Modelagem Dimensional: Estrela e Floco de Neve)

from pyspark.sql.functions import col, lit

# --- 1. CONFIGURAÇÃO E LEITURA DA SILVER ---
SILVER_TABLE = "dev_catalogue.staging_schema.silver_insurance_features"
df_silver = spark.read.table(SILVER_TABLE)

# --- 2. CRIAÇÃO DA TABELA FATO (Fact_Insurance_Cost) ---
FACT_TABLE = "dev_catalogue.staging_schema.gold_fact_insurance_cost"

# A Tabela Fato contém métricas e todas as FKs.
df_fact = df_silver.select(
    col("SK_ID").alias("Insurance_SK_ID"), # PK/FK principal
    col("SK_LOCATION"),
    col("SK_HABIT"),
    col("SK_TIME"),
    col("age"),
    col("bmi"),
    col("children"),
    col("insurance_cost")
)

# Salvar a Tabela Fato
df_fact.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable(FACT_TABLE)

print(f"✅ Tabela FATO CENTRAL criada em: {FACT_TABLE}")
print("---")


# ==========================================================
# 🌟 ESQUEMA ESTRELA (STAR SCHEMA) - Otimizado para Queries Simples (Databricks SQL)
# ==========================================================

# 2.1 DIMENSÃO LOCALIZAÇÃO (Estrela - Desnormalizada)
DIM_LOCATION_STAR_TABLE = "dev_catalogue.staging_schema.gold_dim_location_star"

df_dim_location_star = df_silver.select(
    col("SK_LOCATION"),
    col("region").alias("Region_Name") # Atributos agrupados
).distinct()

df_dim_location_star.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable(DIM_LOCATION_STAR_TABLE)

print(f"✅ Dimensão Estrela (LOCATION) criada em: {DIM_LOCATION_STAR_TABLE}")


# 2.2 DIMENSÃO HÁBITOS (Estrela - Desnormalizada)
DIM_HABIT_STAR_TABLE = "dev_catalogue.staging_schema.gold_dim_habit_star"

df_dim_habit_star = df_silver.select(
    col("SK_HABIT"),
    col("sex").alias("Gender"),
    col("smoker").alias("Smoker_Status") # Atributos agrupados
).distinct()

df_dim_habit_star.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable(DIM_HABIT_STAR_TABLE)

print(f"✅ Dimensão Estrela (HABIT) criada em: {DIM_HABIT_STAR_TABLE}")
print("---")

# ==========================================================
# ❄️ ESQUEMA FLOCO DE NEVE (SNOWFLAKE SCHEMA) - Otimizado para Normalização (Power BI)
# ==========================================================

# 3.1 DIMENSÃO HÁBITOS (Floco de Neve - Normalizada)
DIM_HABIT_SNOW_TABLE = "dev_catalogue.staging_schema.gold_dim_habit_snow"
DIM_SMOKER_STATUS_TABLE = "dev_catalogue.staging_schema.gold_dim_smoker_status"

# Tabela 3.1.a: Sub-Dimensão (Status do Fumante)
df_dim_smoker = df_silver.select(
    col("SK_HABIT").alias("Smoker_SK"),
    col("smoker").alias("Smoker_Status")
).distinct()

df_dim_smoker.write.format("delta").mode("overwrite").saveAsTable(DIM_SMOKER_STATUS_TABLE)

# Tabela 3.1.b: Dimensão Principal (Habit)
# Esta tabela se conecta à FATOS e contém apenas a FK para a Sub-Dimensão
df_dim_habit_snow = df_silver.select(
    col("SK_HABIT"),
    col("sex").alias("Gender"),
    col("SK_HABIT").alias("Smoker_SK") # Usamos a SK_HABIT como FK para Smoker_Status para simplificar
).distinct()

df_dim_habit_snow.write.format("delta").mode("overwrite").saveAsTable(DIM_HABIT_SNOW_TABLE)

print(f"✅ Dimensão Floco de Neve (HABIT) e Sub-Dimensão (SMOKER) criadas.")
print("-" * 50)
print(f"FLUXO DE DADOS COMPLETO: BRONZE -> SILVER -> GOLD (STAR/SNOWFLAKE)")

# Exemplo de como o Snowflake se parece:
# Fact_Insurance_Cost -> Dim_Habit_Snow -> Dim_Smoker_Status